In [1]:
#imports
import duckdb
import warnings
import pandas as pd

In [ ]:
#Settings
pd.set_option('display.max_columns', None)
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
# create a connection to a file called 'database.db'
con = duckdb.connect('database.db')

In [3]:
#describe podcasts table
con.sql("describe podcasts")

┌────────────────┬─────────────┬─────────┬─────────┬─────────┬───────┐
│  column_name   │ column_type │  null   │   key   │ default │ extra │
│    varchar     │   varchar   │ varchar │ varchar │ varchar │ int32 │
├────────────────┼─────────────┼─────────┼─────────┼─────────┼───────┤
│ podcast_id     │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ itunes_id      │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ slug           │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ itunes_url     │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ title          │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ author         │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ description    │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ average_rating │ DOUBLE      │ YES     │ NULL    │ NULL    │  NULL │
│ ratings_count  │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ scraped_at     │ TIMESTAMP   │ YES     │ NULL    │ NULL    │  NULL │
├─────

In [4]:
#describe podcasts table
con.sql("describe reviews")


┌─────────────┬──────────────────────────┬─────────┬─────────┬─────────┬───────┐
│ column_name │       column_type        │  null   │   key   │ default │ extra │
│   varchar   │         varchar          │ varchar │ varchar │ varchar │ int32 │
├─────────────┼──────────────────────────┼─────────┼─────────┼─────────┼───────┤
│ podcast_id  │ VARCHAR                  │ YES     │ NULL    │ NULL    │  NULL │
│ title       │ VARCHAR                  │ YES     │ NULL    │ NULL    │  NULL │
│ content     │ VARCHAR                  │ YES     │ NULL    │ NULL    │  NULL │
│ rating      │ INTEGER                  │ YES     │ NULL    │ NULL    │  NULL │
│ author_id   │ VARCHAR                  │ YES     │ NULL    │ NULL    │  NULL │
│ created_at  │ TIMESTAMP WITH TIME ZONE │ YES     │ NULL    │ NULL    │  NULL │
└─────────────┴──────────────────────────┴─────────┴─────────┴─────────┴───────┘

In [5]:
#Look into data for reviews
con.sql("select * from reviews limit 5")

┌──────────────────────┬──────────────────────┬──────────────────────┬────────┬─────────────────┬──────────────────────┐
│      podcast_id      │        title         │       content        │ rating │    author_id    │      created_at      │
│       varchar        │       varchar        │       varchar        │ int32  │     varchar     │ timestamp with tim…  │
├──────────────────────┼──────────────────────┼──────────────────────┼────────┼─────────────────┼──────────────────────┤
│ 52e3d2c4fab4e80a8b…  │ Argh                 │ What the heck is t…  │      1 │ a23655ce5565f32 │ 2019-04-30 21:24:0…  │
│ 52e3d2c4fab4e80a8b…  │ Miss gross overrat…  │ The Howard stern i…  │      1 │ e5d461c5f91c8d2 │ 2019-06-07 10:25:3…  │
│ 52e3d2c4fab4e80a8b…  │ I love Terri and F…  │ People get really …  │      5 │ 806cc75442c8f3d │ 2019-05-15 05:42:5…  │
│ 52e3d2c4fab4e80a8b…  │ Fresh Air subscrip…  │ I’ve been a subscr…  │      4 │ 3e1e7e8417a3e85 │ 2019-05-05 22:39:1…  │
│ 52e3d2c4fab4e80a8b…  │ Climate

In [6]:
#Look into data for reviews that have an average rating
rated_podcasts = con.sql("select title,average_rating,ratings_count from podcasts where average_rating >= 0")
rated_podcasts

┌─────────────────────────────────────────────────────────────────────────┬────────────────┬───────────────┐
│                                  title                                  │ average_rating │ ratings_count │
│                                 varchar                                 │     double     │    varchar    │
├─────────────────────────────────────────────────────────────────────────┼────────────────┼───────────────┤
│ Rahdo Talks Through                                                     │            4.6 │ 60            │
│ OriginALL                                                               │            3.7 │ 3             │
│ The Big Podcast With Shaq                                               │            4.6 │ 1.3K          │
│ Galaktacus                                                              │            5.0 │ 20            │
│ No Extra Words one person's search for story                            │            5.0 │ 11            │
│ The Talking Pictu

In [7]:
total_podcast_ratings = 0
rated_podcasts_df = rated_podcasts.df()
podcast_ratings = []
for (colname,colval) in rated_podcasts_df.iteritems():
    if colname == "ratings_count":
        for x in colval:
            if  "K" not in x:
                total_podcast_ratings += int(x)
                podcast_ratings.append(int(x))
            else:
                formatted = float(x[0:x.find('K')])
                total_podcast_ratings += (1000 * formatted)   
                podcast_ratings.append(int(1000 * formatted))
rated_podcasts_df.drop("ratings_count", axis = 1, inplace = True)
rated_podcasts_df["ratings_count"] = podcast_ratings

In [8]:
#First five values in rated podcasts data frame
rated_podcasts_df.head()


,title,average_rating,ratings_count
0,Rahdo Talks Through,4.6,60
1,OriginALL,3.7,3
2,The Big Podcast With Shaq,4.6,1300
3,Galaktacus,5.0,20
4,No Extra Words one person's search for story,5.0,11


In [9]:
#Podcasts with top 5 ratings count
rated_podcasts_df.sort_values('ratings_count',ascending=False).head(5)

,title,average_rating,ratings_count
6022,Pardon My Take,4.9,56000
12424,Pod Save America,4.7,51000
57663,Stuff You Should Know,4.6,39000
12588,Small Town Murder,4.9,31000
14355,Wait Wait...Don't Tell Me!,4.7,21000


In [10]:
#Podcasts with top 5 average rating
rated_podcasts_df.sort_values('average_rating',ascending=False).head(5)

,title,average_rating,ratings_count
41494,Cerealcast,5.0,1
42157,Global Denmark Podcast,5.0,9
40733,Faking It: An SFX Podcast,5.0,4
73774,NSVille,5.0,3
40735,Under The First Floor,5.0,5
